In [1]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import scipy as scp
from scipy import sparse

NUM_SAMPLES = 200
SMALL_VAL = -100000000
ITER_THRESH = 5

In [2]:
#A function to get the 20 newsgroup data
def get_data():
    #news_groups_all = fetch_20newsgroups(subset='all')
    #news_data = news_groups_all.data

    #Load in the vectorized news group data from scikit-learn package
    news = fetch_20newsgroups(subset='all')
    all_data = np.array(news.data)
    all_targets = np.array(news.target)
    class_names = news.target_names

    #Set class pairings as described in the multiview clustering paper
    view1_classes = ['comp.graphics','rec.motorcycles', 'sci.space', 'rec.sport.hockey', 'comp.sys.ibm.pc.hardware']
    view2_classes = ['rec.autos', 'sci.med','misc.forsale', 'soc.religion.christian','comp.os.ms-windows.misc']
    
    #Create lists to hold data and labels for each of the 5 classes across 2 different views
    labels =  [num for num in range(len(view1_classes)) for _ in range(NUM_SAMPLES)]
    labels = np.array(labels)
    view1_data = list()
    view2_data = list()
    
    #Randomly sample 200 items from each of the selected classes in view1
    for ind in range(len(view1_classes)):
        class_num = class_names.index(view1_classes[ind])
        class_data = all_data[(all_targets == class_num)]
        indices = np.random.choice(class_data.shape[0], NUM_SAMPLES)
        view1_data.append(class_data[indices])
    view1_data = np.concatenate(view1_data)
   
        
    #Randomly sample 200 items from each of the selected classes in view2
    for ind in range(len(view2_classes)):
        class_num = class_names.index(view2_classes[ind])
        class_data = all_data[(all_targets == class_num)]
        indices = np.random.choice(class_data.shape[0], NUM_SAMPLES)
        view2_data.append(class_data[indices])  
    view2_data = np.concatenate(view2_data)
    
    #Vectorize the data
    vectorizer = CountVectorizer()
    view1_data = vectorizer.fit_transform(view1_data)
    view2_data = vectorizer.fit_transform(view2_data)

    #Shuffle and normalize vectors
    shuffled_inds = np.random.permutation(NUM_SAMPLES * len(view1_classes))
    view1_data = sparse.vstack(view1_data)
    view2_data = sparse.vstack(view2_data)
    view1_data = np.array(view1_data[shuffled_inds].todense())
    view2_data = np.array(view2_data[shuffled_inds].todense())
    labels = labels[shuffled_inds]

    return view1_data, view2_data, labels


In [3]:
v1_data, v2_data, labels = get_data()

In [4]:
def log_sum_exp(log_likes):
    a_vals =  np.max(log_likes, axis = 0)
    likes = np.exp(log_likes - a_vals)
    log_sum = a_vals + np.log(np.sum(likes, axis = 0))
    return log_sum
    
def compute_posterior(data, w_probs, alphas, k):
    log_likes = list()
    for ind in range(k):
        #Compute the likelihood
        log_like = np.sum(data * np.log(w_probs[ind]), axis = 1)
        log_likes.append(log_like)  
    log_likes = np.vstack(log_likes)
    #log_prior = np.log(alphas).reshape(-1, 1)
    log_prior = np.log(np.array([0.2, 0.2, 0.2, 0.2, 0.2])).reshape(-1,1)
    numer = log_prior + log_likes
    denoms = log_sum_exp(numer)
    log_post = numer - denoms
    
    #Compute the denominator of posterior
    posterior = np.exp(log_post)
    print(np.sum(posterior, axis = 0)[0])
    log_likes = np.sum(denoms)
    
    return posterior, log_likes

def compute_posterior2(data, w_probs, alphas, k):
    likes = list()
    for ind in range(k):
        like = data * np.log(w_probs[ind])
        like = np.sum(like, axis=1)
        likes.append(like)  
    likes = np.vstack(likes)
    likes_p = likes * alphas.reshape((-1, 1))
    likes_sum = np.sum(likes_p, axis=1).reshape((-1, 1))
    likes_sum[likes_sum == 0] = 1
    posterior = likes_p / likes_sum
    log_like = np.sum(np.log2(likes_sum))
    return posterior, log_like

def iterate(data, posteriors, k):
    
    #For each of the mixture components, compute model params
    w_probs = list()
    print(posteriors.shape)
    for ind in range(k):
        numer = data * posteriors[ind].reshape((-1, 1))
        print(numer.shape)
        numer = 1 + np.sum(numer, axis=0)
        print(numer.shape)
        denom = np.sum(numer)
        if(denom == 0):
            denom = 1
        probs = numer/denom
        w_probs.append(probs)
    w_probs = np.vstack(w_probs)
    alphas = np.mean(posteriors, axis=1)
    
    #Compute new posterior
    new_posteriors, log_like = compute_posterior(data, w_probs, alphas, k)
    return w_probs, alphas, new_posteriors, log_like

def final_clusters(posteriors):
    f_clusters = np.argmax(posteriors, axis = 0)
    return f_clusters

def compute_entropy(partitions, labels, k, num_classes):
    
    total_entropy = 0
    num_examples = partitions.shape[0]
    for part in range(k):
        labs = labels[partitions == part]
        if(labs.shape[0] == 0):
            continue
        part_size = labs.shape[0]
        part_entropy = 0
        for cl in range(num_classes):
            prop = np.sum(labs == cl) * 1.0 / part_size
            ent = 0
            if(prop != 0):
                ent = - prop * np.log2(prop)
            part_entropy += ent
        part_entropy = part_entropy * part_size / num_examples
        total_entropy += part_entropy
    return total_entropy
    

In [5]:
#The main kmeans clustering algorithm
def multinomial(v_data, labels, k = 5):
    
    v_data = np.concatenate(v_data, axis = 1)
    #Initialize cluster centers, partitions, and loop params
    w_probs = np.random.random((k, v_data.shape[1]))
    
    w_probs /= np.sum(w_probs, axis=1).reshape((-1, 1))    
    alphas = (1/k) * np.ones((k,))
    
    posterior, log_likes = compute_posterior(v_data, w_probs, alphas, k)
    f_clusters = np.argmax(posterior, axis = 0)
    #for ind in range(k):
        #print(np.sum(f_clusters == ind))
    objective = SMALL_VAL
    iter_stall = 0
    iter_num = 0
    entropy = 0
    
    while(iter_stall < ITER_THRESH):
        iter_num += 1
        view = (iter_num + 1) % 2
        
        #Switch partitions, Maximization, and Expectation
        w_probs, alphas, posterior, log_like = iterate(v_data, posterior, k)
        iter_stall += 1
        #Recompute objective function
        if(log_like > objective):
            objective = log_like
            iter_stall = 0
        
        #Obtain evaluation metrics
        f_clusters = np.argmax(posterior, axis = 0)
        for ind in range(k):
            print(np.sum(f_clusters == ind))
        entropy = compute_entropy(f_clusters, labels, k, 5)
        print(entropy)

    return entropy
        

In [6]:
ent = multinomial([v1_data, v2_data], labels, 5)
print(ent)

1.0
(5, 1000)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
1.0
325
0
431
51
193
2.2702071932240906
(5, 1000)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
1.0
339
0
427
26
208
2.229405061127002
(5, 1000)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
1.0
348
0
419
19
214
2.2089592794010926
(5, 1000)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
1.0
349
0
419
14
218
2.197198059639268
(5, 1000)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
1.0
350
0
420
7
223
2.1832741573244316
(5, 1000)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
(1000, 45947)
(45947,)
1.0
351
0
423
6
220
2.1793037264356694
(5, 1000)
